In [1]:
from typing import List
from dao.email import DAOEmailGenerated, DAORealEmail

from dao.attribute import DAOAttribute

from ml.model_training import evaluate_models
from ml.data_preparation import find_significant_features

[nltk_data] Downloading package stopwords to /home/pawel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pawel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package pl196x to /home/pawel/nltk_data...
[nltk_data]   Package pl196x is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pawel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from models.attribute import AttributeInDB

dao_generated_emails: DAOEmailGenerated = DAOEmailGenerated()
dao_real_emails: DAORealEmail = DAORealEmail()
dao_attribute: DAOAttribute = DAOAttribute()


all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({})
all_generated: List[AttributeInDB] = dao_attribute.find_many_by_query({'is_generated': True})
all_real: List[AttributeInDB] = dao_attribute.find_many_by_query({'is_generated': False})

generated_personal: List[AttributeInDB] = dao_attribute.find_many_by_query({"is_generated": True, 'is_personal': True})
real_personal: List[AttributeInDB] = dao_attribute.find_many_by_query({"is_generated": False, 'is_personal': True})

generated_personal = [attribute for attribute in generated_personal if attribute is not None]
real_personal = [attribute for attribute in real_personal if attribute is not None]

pl_generated = [attribute for attribute in all_generated if attribute.language == 'pl']
pl_real = [attribute for attribute in all_real if attribute.language == 'pl']
en_generated = [attribute for attribute in all_generated if attribute.language == 'en']
en_real = [attribute for attribute in all_real if attribute.language == 'en']

pl_generated_personal = [attribute for attribute in generated_personal if attribute.language == 'pl']
pl_real_personal = [attribute for attribute in real_personal if attribute.language == 'pl']
en_generated_personal = [attribute for attribute in generated_personal if attribute.language == 'en']
en_real_personal = [attribute for attribute in real_personal if attribute.language == 'en']

In [43]:
generated = pl_generated_personal
real = pl_real_personal
num_of_features = 10

data = [[x.to_flat_dict_normalized(), 1]for x in generated]
data += [[x.to_flat_dict_normalized(), 0] for x in real]
# replace None with 0
for i in range(len(data)):
    for key in data[i][0].keys():
        if data[i][0][key] is None:
            data[i][0][key] = 0
            
significant_features = find_significant_features(data)
features_to_exclude = significant_features[num_of_features:]

data = [[x.to_flat_dict_normalized(exclude=features_to_exclude), 1]for x in generated]
data += [[x.to_flat_dict_normalized(exclude=features_to_exclude), 0] for x in real]
# replace None with 0
for i in range(len(data)):
    for key in data[i][0].keys():
        if data[i][0][key] is None:
            data[i][0][key] = 0

/mnt/d/Dev/Github/anti-gpt-checker/ml/data_preparation.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [40]:
data[0][0]

{'punctuation_per_sentence': 1.6666666666666667}

In [41]:
model_valuation = evaluate_models(data)

/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [42]:
model_valuation

{'Decision Tree': {'accuracy': 0.8413793103448276,
  'precision': 0.8121212121212121,
  'recall': 0.8993288590604027,
  'f1_score': 0.8535031847133758,
  'roc_auc': 0.8750773478033224},
 'Random Forest': {'accuracy': 0.8517241379310345,
  'precision': 0.8154761904761905,
  'recall': 0.9194630872483222,
  'f1_score': 0.8643533123028391,
  'roc_auc': 0.885739444999762},
 'MLP Classifier': {'accuracy': 0.8413793103448276,
  'precision': 0.7877094972067039,
  'recall': 0.9463087248322147,
  'f1_score': 0.8597560975609756,
  'roc_auc': 0.8821695463848827},
 'K-Nearest Neighbors': {'accuracy': 0.8137931034482758,
  'precision': 0.8368794326241135,
  'recall': 0.7919463087248322,
  'f1_score': 0.8137931034482758,
  'roc_auc': 0.885977438240754},
 'Support Vector Classifier': {'accuracy': 0.7965517241379311,
  'precision': 0.7227722772277227,
  'recall': 0.9798657718120806,
  'f1_score': 0.8319088319088319,
  'roc_auc': 0.8639154648007996},
 'AdaBoost': {'accuracy': 0.8482758620689655,
  'prec

In [26]:
len(data)

965